In [6]:
import pandas as pd
tour_place_df = pd.read_csv('./data/tour_place.csv')
cafe_df = pd.read_csv('./data/cafe_final.csv')

In [2]:
tour_place_df.head

,Unnamed: 0,name,address,latitude,longitude
0,0,(주)제주해양레저 투명카약,제주특별자치도 제주시 애월읍 애월리 2490-1,33.462353,126.31049
1,1,1100고지(휴게소),제주특별자치도 서귀포시 색달동 산 1-3,33.339912,126.43682
2,2,1100고지습지,제주특별자치도 서귀포시 색달동 산 1-2,33.357500,126.46292
3,3,1112도로,제주특별자치도 제주시 봉개동 산 78-1,33.439373,126.62845
4,4,4·3해원방사탑,제주특별자치도 제주시 일도2동 837-20,33.505470,126.53451


In [7]:
cafe_df.head()

,Unnamed: 0,addrbunho,address,cafe,cafe_id,image_url,franchise,final_address,lat,lng
0,0,294-24,제주특별자치도 서귀포시 서귀동,여정다방,11877497,NaN,0.0,제주특별자치도 서귀포시 서귀동 294-24,33.248312,126.561575
1,1,475-8,제주특별자치도 제주시 외도일동,아장아장 베이비카페,1263620627,http://t1.daumcdn.net/place/83909D95B967478D80...,0.0,제주특별자치도 제주시 외도일동 475-8,33.489895,126.430688
2,2,175-6,제주특별자치도 서귀포시 대정읍 보성리,소소희,368243275,http://t1.daumcdn.net/place/8395E365EA5B4F04BB...,0.0,제주특별자치도 서귀포시 대정읍 보성리 175-6,33.274976,126.278127
3,3,1325,제주특별자치도 제주시 건입동,항구다방,1571310600,NaN,0.0,제주특별자치도 제주시 건입동 1325,33.516313,126.528937
4,4,437-5,제주특별자치도 서귀포시 성산읍 삼달리,두모악무인찻집,13571332,http://t1.daumcdn.net/place/2535E720C72A4A5B9D...,0.0,제주특별자치도 서귀포시 성산읍 삼달리 437-5,33.371935,126.854377


In [9]:
import geopy.distance

In [16]:
def distance_cafe_place(place, cafe):
    place_pos = place[['latitude', 'longitude']].values
    cafe_pos = cafe[['lat', 'lng']].values
    return geopy.distance.vincenty(place_pos, cafe_pos).m

distance_cafe_place(tour_place_df.iloc[0], cafe_df.iloc[0])

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  after removing the cwd from sys.path.


33312.562679877316

In [11]:
place_df = tour_place_df[~tour_place_df.latitude.isna()]
place_df.shape

(1102, 5)

In [12]:
existed_cafe_df = cafe_df[~cafe_df['lat'].isna()]
cafe_df.shape, existed_cafe_df.shape

((3407, 10), (3381, 10))

In [19]:
distance_cafe_place(place_df.iloc[0], existed_cafe_df.iloc[0])

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  after removing the cwd from sys.path.


33312.562679877316

In [21]:
def distance_cafe_places(place, existed_cafe_df):
    return [distance_cafe_place(place, existed_cafe_df.iloc[i]) for i in range(existed_cafe_df.shape[0])]
place  = place_df.iloc[0]
distanced_cafe = existed_cafe_df.assign(distance=distance_cafe_places(place, existed_cafe_df))
distanced_cafe.head(1)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  after removing the cwd from sys.path.


,Unnamed: 0,addrbunho,address,cafe,cafe_id,image_url,franchise,final_address,lat,lng,distance
0,0,294-24,제주특별자치도 서귀포시 서귀동,여정다방,11877497,NaN,0.0,제주특별자치도 서귀포시 서귀동 294-24,33.248312,126.561575,33312.56268


In [27]:
recommended_places = []

for i in range(place_df.shape[0]):
    place = place_df.iloc[i]
    print(i, place['name'])
    distanced_place_df = existed_cafe_df.assign(distance=distance_cafe_places(place, existed_cafe_df))
    places = distanced_place_df.sort_values(by=['distance'])[['cafe', 'distance']][:10].values
    recommended_places.append(places)

cafe_with_place_df = place_df.assign(places=recommended_places)
cafe_with_place_df.to_csv('./cafe_with_place.csv')

0 (주)제주해양레저 투명카약


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  after removing the cwd from sys.path.


1 1100고지(휴게소)
2 1100고지습지
3 1112도로
4 4·3해원방사탑
5 5.16도로숲터널
6 9.81 파크
7 9연대 본부 옛터
8 SM 디지털아트 뮤지엄
9 THE WE
10 가마오름
11 가마오름 진지동굴
12 가메오름
13 가문이오름(감은이오름)
14 가새기오름
15 가세오름
16 가파도
17 가파도 마라도 정기여객선
18 가파초등학교 마라분교장
19 각시바우오름
20 간세공방
21 감귤박물관
22 감귤상웨빵 체험
23 감수굴 밭담길
24 갑선이오름
25 갑자기 히어로즈 벽화길
26 갓전시관
27 강병대교회
28 강정어촌계
29 강정천유원지
30 강창학 종합경기장
31 개오름
32 개오리오름
33 갤러리2 중선농원
34 갯깍주상절리
35 거린오름 (북오름)
36 거문오름(UNESCO 세계자연유산)
37 거문오름승마체험장
38 거믄여해안경승지
39 거슨새미오름
40 거인의 정원
41 거친오름
42 건강과성박물관
43 건강장수마을 산양리 농어촌체험마을
44 걸매생태공원
45 걸매예술마을
46 걸서악 (동걸세, 걸세오름, 걸시오름)
47 걸서악(서걸세,걸세오름)
48 걸서악오름
49 걸시오름
50 검멀레동굴
51 검멀레해변
52 게우지코지
53 고근산(고공산)
54 고내리 다락쉼터
55 고내리포구
56 고내봉(고내오름,고니오름,망오름)
57 고마워토토 아뜰리에
58 고사리숲
59 고산일과해안도로
60 고살리숲길
61 고흐의 정원
62 곤물
63 곤흘동(곤을동) 마을터
64 골머리 (洞山, 笏頭岳, 아흔아홉골)
65 골프탕탕
66 공천포
67 공천포구
68 곶자왈
69 과오름(큰오름, 샛오름, 말젯오름)
70 곽지레저 & 서프나우
71 곽지패총
72 곽지해수욕장
73 곽지해수욕장야영장
74 관덕정
75 관음사
76 관음사 군 주둔지 옛터
77 관음사 템플스테이
78 광령귤나무
79 광이오름 (괭이오름, 肝列岳)
80 광치기해변
81 교래자연휴양림
82 구엄리돌염전
83 구엄포구
84 구좌 하도어촌계
85 구좌해안로
86 국립제주박물관
87 군산(군산

695 우도올레보트
696 우도잠수함
697 웃밤(웃바매기,웃밤오름)
698 웃세오름(누운오름)
699 웃세오름(족은오름)
700 원당봉
701 원동
702 원수악
703 원앙카라반캠핑장앤야영장
704 원앙폭포
705 월나봉(도래오름)
706 월대천
707 월라봉
708 월라봉(효돈)
709 월라사
710 월령선인장군락지
711 월령포구
712 월산봉
713 월정리 카약체험장
714 월정리해변
715 월정리해안도로
716 월평포구
717 위미리 동백나무군락
718 위미항
719 윈드1947 카트 테마파크
720 윗세오름(웃세오름)
721 유건에오름
722 유민미술관
723 육시우영
724 이니스프리제주하우스
725 이달봉
726 이덕구 산전
727 이랜드뮤지엄
728 이로이로공방
729 이상한 나라의 앨리스
730 이스렁오름
731 이중섭거리
732 이중섭거주지
733 이중섭미술관
734 이편한치과
735 이호-내도 해안도로
736 이호털보네배낚시
737 이호테우말등대
738 이호테우해수욕장
739 익스트림제주카약올레
740 일출랜드
741 임문숙 가족 헛묘
742 입산봉(삿갓오름)
743 자구네포구
744 자구리공원
745 자리왓
746 자배봉(자배오름,망오름)
747 자연인
748 장생의 숲길
749 장전리 벚꽃축제길
750 재밋섬파크
751 저지 예술인 마을
752 저지오름(닥몰오름,새오름)
753 전농로 서사라문화거리
754 전망대(종달리)
755 절물오름(큰대나)
756 절부암
757 점보빌리지
758 정모시쉼터
759 정물오름
760 정방폭포
761 정의향교
762 정의현 일관헌
763 정주항
764 제라진 제주오프로드 사파리
765 제원한의원
766 제주 더팔팔농장
767 제주 메이 더 호텔 카라반빌리지
768 제주 삼양동 유적
769 제주 아라고나이트 온천
770 제주 이호털보 배낚시
771 제주 카카오캠핑
772 제주 패러글라이딩 스쿨
773 제주4·3평화공원
774 제주공룡랜드
775 제주곶자왈도립공원
776 제주교육박물관
777 제주국제공항

In [62]:
cafe_with_place_df.head()

,Unnamed: 0,name,address,latitude,longitude,places
0,0,(주)제주해양레저 투명카약,제주특별자치도 제주시 애월읍 애월리 2490-1,33.462353,126.31049,"[[투썸플레이스 제주애월한담점, 50.973390746162224], [썬셋클리프,..."
1,1,1100고지(휴게소),제주특별자치도 서귀포시 색달동 산 1-3,33.339912,126.43682,"[[효월, 4704.594733488161], [티스토리라원, 4997.796645..."
2,2,1100고지습지,제주특별자치도 서귀포시 색달동 산 1-2,33.357500,126.46292,"[[아르크애월, 6083.009175565157], [효월, 7132.5685120..."
3,3,1112도로,제주특별자치도 제주시 봉개동 산 78-1,33.439373,126.62845,"[[클라우드베이, 1359.3669049994803], [카페퐁낭, 1659.081..."
4,4,4·3해원방사탑,제주특별자치도 제주시 일도2동 837-20,33.505470,126.53451,"[[티포이, 178.26818441052248], [디씨펫랜드, 204.977613..."


In [ ]:
recommended_place2 = []
recommended_distance2=[]
place2 = []
for i in range(10):
    distance = cafe_with_place_df['places'][0][i][1]
    name = cafe_with_place_df['places'][0][i][0]
    if distance <= 1000:
        recommended_place2.append(name)
        recommended_distance2.append(distance)

cafe_with_place_df['name'] = list(recommended_place2)  

In [24]:
cafe_distance_tourplace = pd.read_csv('./cafe_distance_tourplace.csv')

In [25]:
cafe_distance_tourplace.head()

,Unnamed: 0,Unnamed: 0.1,addrbunho,address,cafe,cafe_id,image_url,franchise,final_address,lat,lng,places
0,0,0,294-24,제주특별자치도 서귀포시 서귀동,여정다방,11877497,NaN,0.0,제주특별자치도 서귀포시 서귀동 294-24,33.248312,126.561575,[['서귀포 펍크롤' 68.25085901230827]\n ['이중섭거리' 247....
1,1,1,475-8,제주특별자치도 제주시 외도일동,아장아장 베이비카페,1263620627,http://t1.daumcdn.net/place/83909D95B967478D80...,0.0,제주특별자치도 제주시 외도일동 475-8,33.489895,126.430688,[['넥슨컴퓨터박물관' 445.75820272505644]\n ['연대포구' 558...
2,2,2,175-6,제주특별자치도 서귀포시 대정읍 보성리,소소희,368243275,http://t1.daumcdn.net/place/8395E365EA5B4F04BB...,0.0,제주특별자치도 서귀포시 대정읍 보성리 175-6,33.274976,126.278127,[['제주곶자왈도립공원' 1077.9690441462]\n ['노리매공원' 1216...
3,3,3,1325,제주특별자치도 제주시 건입동,항구다방,1571310600,NaN,0.0,제주특별자치도 제주시 건입동 1325,33.516313,126.528937,[['산지천갤러리' 139.86832578003313]\n ['김만덕기념관' 145...
4,4,4,437-5,제주특별자치도 서귀포시 성산읍 삼달리,두모악무인찻집,13571332,http://t1.daumcdn.net/place/2535E720C72A4A5B9D...,0.0,제주특별자치도 서귀포시 성산읍 삼달리 437-5,33.371935,126.854377,[['김영갑갤러리 두모악' 23.140257276500627]\n ['고흐의 정원'...


In [28]:
cafe_with_place = pd.read_csv('./cafe_with_place.csv')

In [30]:
cafe_with_place.shape

(1102, 7)

In [64]:
cafe_with_place['places'][0]

"[['투썸플레이스 제주애월한담점' 50.973390746162224]\n ['썬셋클리프' 61.152342392299964]\n ['어보브앤비욘드' 61.152342392299964]\n ['바바' 63.91243191595533]\n ['봄날' 96.48666542605073]\n ['모립' 100.04606211530498]\n ['돌돌' 119.97327564055492]\n ['팜파네' 119.97327564055492]\n ['커플' 126.95443013982307]\n ['이뻬하우스카페' 144.23002890525783]]"

In [61]:
cafe_with_place['places']

0       [['투썸플레이스 제주애월한담점' 50.973390746162224]\n ['썬셋클...
1       [['효월' 4704.594733488161]\n ['티스토리라원' 4997.796...
2       [['아르크애월' 6083.009175565157]\n ['효월' 7132.5685...
3       [['클라우드베이' 1359.3669049994803]\n ['카페퐁낭' 1659....
4       [['티포이' 178.26818441052248]\n ['디씨펫랜드' 204.977...
                              ...                        
1097    [['상효원수목원 카페테리아' 3774.7074606710316]\n ['카페테리아...
1098    [['알라스카인제주 용담점' 52594.08569104601]\n ['해적카페' 5...
1099    [['아르크애월' 5890.65924871573]\n ['마실온다메셀프카페' 792...
1100    [['스프링데일골프앤리조트 통나무카페' 2456.079128191961]\n ['카...
1101    [['홀리가든' 874.039989656354]\n ['꼬닥꼬닥' 960.57622...
Name: places, Length: 1102, dtype: object

In [34]:
recommended_place2 = []
recommended_distance2=[]
place2 = []
for i in range(10):
    distance = cafe_with_place['places'][0][i][1]
    name = cafe_with_place['places'][0][i][0]
    if distance <= 1000:
        recommended_place2.append(name)
        recommended_distance2.append(distance)

# cafe_with_place['name'] = list(recommended_place2)  

IndexError: string index out of range